In [796]:
import requests
import pandas as pd
import re
from datetime import datetime, timedelta
from bs4 import BeautifulSoup as bs
import numpy as np
import time

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept-Language': 'fr-FR,fr;q=0.9,en;q=0.8',
    'Referer': 'https://www.allocine.fr/'
}

In [798]:
columns = ['Rang', 'NbSemaine', 'Titre', 'Entrées', 'Variation', 'Cumul', 'Budget', 'NbCopies', 'MoyCopies']
data = []

for i in range(52) :
    if i < 9 :
        url = "http://www.cine-directors.net/box/2025/boxoff0"+str(i+1)+".html"
    else :
        url = "http://www.cine-directors.net/box/2025/boxoff"+str(i+1)+".html"
    page = requests.get(url, headers=headers)
    soup = bs(page.content, 'lxml')
    tbody = soup.find('tbody').find_next('tbody')
    rows = tbody.find_all('tr')[4:]  # Ignorer les 3 premières lignes
    for row in rows:
        cells = row.find_all('td')
        if len(cells) == 9:
            row_data = {col: cells[i].get_text().strip() for i, col in enumerate(columns)}
            
            text = tbody.find('table').find('td').get_text().strip()
            cleaned = ' '.join(text.split())
            row_data['Date'] = re.search(r'du\s+(.+?)\)', cleaned).group(1)
            data.append(row_data)
BoxOffice = pd.DataFrame(data)

In [801]:
#On va devoir nettoyer le fichier
#Par exemple:
# Dans la colonne Entrées/NbCopies/MoyCopies, il y a des espaces dans les nombres
# Dans la colonne Cumul, il y a des virgules dans les nombres
# Dans la colonne Budget, nous avons le budget en million, soit en euros ou en dollars
BoxOffice.head()

,Rang,NbSemaine,Titre,Entrées,Variation,Cumul,Budget,NbCopies,MoyCopies,Date
0,1,3,Mufasa : Le roi lion,986 000,- 28 %,"3,574",200 M$,951,1 037,1er au 7 janvier 2025
1,2,2,"Sonic 3, le film",655 000,- 47 %,"1,879",122 M$,772,848,1er au 7 janvier 2025
2,3,6,Vaiana 2,557 000,- 43 %,"7,506",150 M$,1 063,524,1er au 7 janvier 2025
3,4,1,Un ours dans le Jura,453 000,New,"0,453","12,1 M€",580,782,1er au 7 janvier 2025
4,5,6,En fanfare,244 000,- 19 %,"1,854","6,1 M€",642,379,1er au 7 janvier 2025


In [802]:
#Toutes les colonnes sont des objets
BoxOffice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1420 entries, 0 to 1419
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Rang       1420 non-null   object
 1   NbSemaine  1420 non-null   object
 2   Titre      1420 non-null   object
 3   Entrées    1420 non-null   object
 4   Variation  1420 non-null   object
 5   Cumul      1420 non-null   object
 6   Budget     1420 non-null   object
 7   NbCopies   1420 non-null   object
 8   MoyCopies  1420 non-null   object
 9   Date       1420 non-null   object
dtypes: object(10)
memory usage: 111.1+ KB


In [803]:
#Nous allons procéder au nettoyage du DataFrame

#Lorsque le rang est null, on supprime la ligne
BoxOffice = BoxOffice[BoxOffice['Rang']!='']

#Nous remarquons que pour certains films, nous avons des doublons, car des films ont (FT) en fin de titre
BoxOffice = BoxOffice.replace(r' +\(FT\)', '', regex=True)

#On nettoie la colonne NbCopies et on la transforme en int
BoxOffice['NbCopies'] = BoxOffice['NbCopies'].str.replace(' ', '')
BoxOffice['NbCopies']= BoxOffice['NbCopies'].replace(['?','-',''], np.nan)
BoxOffice['NbCopies']= BoxOffice['NbCopies'].astype('Int64')

#On nettoie la colonne MoyCopies et on la transforme en int
BoxOffice['MoyCopies'] = BoxOffice['MoyCopies'].str.replace(' ', '')
BoxOffice['MoyCopies'] = BoxOffice['MoyCopies'].replace(['?','-',''], np.nan)
BoxOffice['MoyCopies'] = BoxOffice['MoyCopies'].astype('Int64')

#On nettoie la colonne Entrées et on la transforme en int
BoxOffice['Entrées'] = BoxOffice['Entrées'].str.replace(' ', '')
BoxOffice['Entrées'] = BoxOffice['Entrées'].replace('77entrées', '77')
BoxOffice['Entrées'] = BoxOffice['Entrées'].replace(['?','-',''], np.nan)
BoxOffice['Entrées'] = BoxOffice['Entrées'].astype('Int64')

#On nettoie la colonne Cumul, on la transforme en int et on multiplie par 1000 pour avoir le bon nombre
#On modifie le cumul d'un film car il y a une coquille sur le site
BoxOffice['Cumul'] = BoxOffice['Cumul'].str.replace(',','')
BoxOffice['Cumul'] = BoxOffice['Cumul'].astype('Int64')*1000
BoxOffice.loc[(BoxOffice['Titre'] == 'Demon slayer : La forteresse infinie - Film 1') & (BoxOffice['Rang'] == '1'),'Cumul']/=10 

#On nettoie la colonne Budget. On fait d'un côté les dollars car il faut transformer la devise en euro.
mask2 = ~BoxOffice['Budget'].str.contains('M\$', regex=True, na=False)
BoxOffice.loc[mask2, 'Budget'] = BoxOffice.loc[mask2, 'Budget'].str.replace(' M\€', '', regex=True).str.replace(',','.').replace('-',np.nan).replace('- M£',np.nan).replace('',np.nan)

mask = BoxOffice['Budget'].str.contains('M\$', regex=True, na=False)
BoxOffice.loc[mask, 'Budget'] = (BoxOffice.loc[mask, 'Budget'].str.replace(' M\$', '', regex=True).replace('-',np.nan).str.replace(',','.'))

BoxOffice['Budget'] = BoxOffice['Budget'].astype(float)
BoxOffice['Budget'] = BoxOffice['Budget']*1000000
BoxOffice.loc[mask, 'Budget'] = BoxOffice.loc[mask, 'Budget'] * 1.20

#On modifie le type du Nombre de semaine
BoxOffice['NbSemaine'] = BoxOffice['NbSemaine'].astype(int)

#On rajoute une colonne où l'on va calculer le ratio entre le nombre total d'entrée d'un film avec son budget
BoxOffice['Ratio'] = BoxOffice['Cumul']/BoxOffice['Budget']

In [806]:
#Nous avons maintenant le DataFrame bien nettoyé
BoxOffice.head()

,Rang,NbSemaine,Titre,Entrées,Variation,Cumul,Budget,NbCopies,MoyCopies,Date,Ratio
0,1,3,Mufasa : Le roi lion,986000,- 28 %,3574000,240000000.0,951,1037,1er au 7 janvier 2025,0.014892
1,2,2,"Sonic 3, le film",655000,- 47 %,1879000,146400000.0,772,848,1er au 7 janvier 2025,0.012835
2,3,6,Vaiana 2,557000,- 43 %,7506000,180000000.0,1063,524,1er au 7 janvier 2025,0.0417
3,4,1,Un ours dans le Jura,453000,New,453000,12100000.0,580,782,1er au 7 janvier 2025,0.037438
4,5,6,En fanfare,244000,- 19 %,1854000,6100000.0,642,379,1er au 7 janvier 2025,0.303934


In [807]:
#Nous avons maintenant les colonnes avec le bon type
BoxOffice.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1368 entries, 0 to 1418
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Rang       1368 non-null   object 
 1   NbSemaine  1368 non-null   int64  
 2   Titre      1368 non-null   object 
 3   Entrées    1367 non-null   Int64  
 4   Variation  1368 non-null   object 
 5   Cumul      1368 non-null   Int64  
 6   Budget     1070 non-null   float64
 7   NbCopies   1361 non-null   Int64  
 8   MoyCopies  1361 non-null   Int64  
 9   Date       1368 non-null   object 
 10  Ratio      1070 non-null   Float64
dtypes: Float64(1), Int64(4), float64(1), int64(1), object(4)
memory usage: 134.9+ KB


In [808]:
#Nous pouvons maintenant répondre à un certain nombre de questions

In [821]:
#Nous pouvons avoir un premier aperçu des statistiques descriptives du DataFrame
BoxOffice.describe()

,NbSemaine,Entrées,Cumul,Budget,NbCopies,MoyCopies,Ratio
count,1368.000000,1367.0,1368.0,1.070000e+03,1361.0,1361.0,1070.0
mean,3.521199,96841.565472,673135.233918,6.023672e+07,416.080088,215.566495,0.045632
std,2.890008,181505.243018,1202736.036565,8.966206e+07,222.18754,274.058308,0.091611
min,1.000000,0.0,6000.0,5.200000e+05,24.0,3.0,0.000479
25%,2.000000,22000.0,94750.0,5.500000e+06,242.0,76.0,0.007415
50%,3.000000,45000.0,255500.0,1.540000e+07,379.0,137.0,0.017456
75%,5.000000,100000.0,680750.0,7.200000e+07,553.0,261.0,0.043865
max,25.000000,2541000.0,14510000.0,4.800000e+08,1377.0,3490.0,1.009615


In [823]:
#Quel est le nombre d'entrées en 2025 ?
print("Le nombre d'entrées en 2025 est de ",BoxOffice['Entrées'].sum())

Le nombre d'entrées en 2025 est de  132382420


In [809]:
#Quels films ont fait le plus d'entrées en une semaine ?
BoxOffice[BoxOffice['Entrées'] != np.nan][['Titre','Entrées','NbSemaine','Date']].sort_values('Entrées', ascending=False).head(10)

,Titre,Entrées,NbSemaine,Date
1367,Avatar : De feu et de cendres,2541000,1,17 au 23 décembre 2025
1393,Avatar : De feu et de cendres,2516000,2,24 au 30 décembre 2025
1283,Zootopie 2,1736000,1,26 novembre au 2 décembre 2025
579,Lilo & Stitch,1349000,2,28 mai au 3 juin 2025
553,Lilo & Stitch,1325000,1,21 au 27 mai 2025
1394,La femme de ménage,1312000,1,24 au 30 décembre 2025
1368,Zootopie 2,1284000,4,17 au 23 décembre 2025
1309,Zootopie 2,1269000,2,3 au 9 décembre 2025
1395,Zootopie 2,1269000,5,24 au 30 décembre 2025
977,The conjuring : l'heure du jugement,1109000,1,10 au 16 septembre 2025


In [810]:
#Quels films ont fait le plus d'entrée en 2025 ?
BoxOffice[BoxOffice['Cumul'] != np.nan][['Titre','Cumul','NbSemaine','Date']].sort_values('Cumul', ascending=False).drop_duplicates(subset='Titre').head(10)
#D'autres films comme Avatar : La Voie de l'eau ou Star Wars sont dans la liste, car ils sont ressortis en 2025, mais ils ont quand même gardé leur nombre d'entrées de leur première année de sortie.

,Titre,Cumul,NbSemaine,Date
1116,Avatar : La voie de l'eau,14510000,1,8 au 14 octobre 2025
239,Vaiana 2,8066000,14,25 février au 4 mars 2025
1395,Zootopie 2,6547000,5,24 au 30 décembre 2025
322,Mufasa : Le roi lion,5233000,14,19 au 25 mars 2025
920,Lilo & Stitch,5137000,14,20 au 26 août 2025
1393,Avatar : De feu et de cendres,5057000,2,24 au 30 décembre 2025
47,L'amour ouf,4901000,13,8 au 14 janvier 2025
446,Star wars : Episode 3 - La revanche des Sith,3452000,1,23 au 29 avril 2025
1059,F1,3290000,14,24 au 30 septembre 2025
46,Gladiator 2,3036000,9,8 au 14 janvier 2025


In [811]:
#Quels films sont restés le plus longtemps au cinéma ?
BoxOffice[['Titre','NbSemaine']].sort_values('NbSemaine', ascending=False).drop_duplicates('Titre').head(10)

,Titre,NbSemaine
550,Conclave,25
131,Emilia Perez,24
134,Les graines du figuier sauvage,20
292,Anora,20
133,L'histoire de Souleymane,17
264,En fanfare,15
1059,F1,14
920,Lilo & Stitch,14
322,Mufasa : Le roi lion,14
239,Vaiana 2,14


In [812]:
#Quels films ont le plus grand nombre de copies en une semaine ?
BoxOffice[BoxOffice['NbCopies'] != np.nan][['Titre','NbCopies','Date']].sort_values('NbCopies', ascending=False).head(10)

,Titre,NbCopies,Date
1395,Zootopie 2,1377,24 au 30 décembre 2025
1368,Zootopie 2,1275,17 au 23 décembre 2025
717,Dragons,1126,2 au 8 juillet 2025 - Fête du cinéma bis
746,Elio,1104,9 au 15 juillet 2025
744,Dragons,1084,9 au 15 juillet 2025
662,Lilo & Stitch,1083,18 au 24 juin 2025
443,"Minecraft, le film",1081,23 au 29 avril 2025
898,Les bad guys 2,1071,20 au 26 août 2025
2,Vaiana 2,1063,1er au 7 janvier 2025
774,Elio,1060,16 au 22 juillet 2025


In [813]:
#Quels films ont le plus moins de copies ?
BoxOffice[BoxOffice['NbCopies'] != np.nan][['Titre','NbCopies','Date']].sort_values('NbCopies', ascending=False).tail(10)

,Titre,NbCopies,Date
133,L'histoire de Souleymane,67,29 janvier au 4 février 2025
497,Ne zha 2,61,30 avril au 6 mai 2025
652,Le rendez-vous de l'été,56,11 au 17 juin 2025
43,Personne n'y comprend rien,52,8 au 14 janvier 2025
626,Zion,49,4 au 10 juin 2025
624,Jardin d'été,26,4 au 10 juin 2025
686,Thunderbolts*,26,18 au 24 juin 2025
684,Sinners,26,18 au 24 juin 2025
134,Les graines du figuier sauvage,25,29 janvier au 4 février 2025
685,Until dawn,24,18 au 24 juin 2025


In [815]:
#Quels films ont le plus grand nombre de moyenne de copies ?
BoxOffice[BoxOffice['MoyCopies'] != np.nan][['Titre','MoyCopies','Date']].sort_values('MoyCopies', ascending=False).head(10)

,Titre,MoyCopies,Date
1367,Avatar : De feu et de cendres,3490,17 au 23 décembre 2025
1393,Avatar : De feu et de cendres,3118,24 au 30 décembre 2025
1283,Zootopie 2,2691,26 novembre au 2 décembre 2025
977,The conjuring : l'heure du jugement,2526,10 au 16 septembre 2025
553,Lilo & Stitch,2246,21 au 27 mai 2025
579,Lilo & Stitch,2212,28 mai au 3 juin 2025
1394,La femme de ménage,2140,24 au 30 décembre 2025
1309,Zootopie 2,1888,3 au 9 décembre 2025
389,Zion,1833,9 au 15 avril 2025
689,F1,1735,25 juin au 1er juillet 2025


In [818]:
#Quels films ont le meilleur ratio Entrées/Budget ?
BoxOffice[['Titre','Cumul','Budget','Ratio']].sort_values('Ratio', ascending = False).drop_duplicates(subset='Titre').head(10)

,Titre,Cumul,Budget,Ratio
378,A bicyclette !,525000,520000.0,1.009615
1280,Sacré-coeur,455000,690000.0,0.65942
1194,Un simple accident,620000,1000000.0,0.62
264,En fanfare,2583000,6100000.0,0.423443
1398,Le chant des forêts,409000,1200000.0,0.340833
160,Vingt dieux,897000,2800000.0,0.320357
1025,La nuit des clowns,306000,1200000.0,0.255
1170,Sirat,673000,3000000.0,0.224333
133,L'histoire de Souleymane,582000,2600000.0,0.223846
1363,La femme le plus riche du monde,900000,4200000.0,0.214286


In [819]:
#Quels films ont les pire ratio Entrées/Budget ?
BoxOffice[['Titre','Cumul','Budget','Ratio']].sort_values('Ratio').drop_duplicates(subset='Titre').head(10)

,Titre,Cumul,Budget,Ratio
456,Ne zha 2,46000,96000000.0,0.000479
258,In the lost lands,32000,66000000.0,0.000485
1079,A big bold beautiful journey,35000,54000000.0,0.000648
1094,Tron : Ares,200000,216000000.0,0.000926
664,Elio,168000,180000000.0,0.000933
1191,Smashing machine,57000,60000000.0,0.00095
82,Better man,142000,132000000.0,0.001076
498,Star wars : Episode 3 - La revanche des Sith,149000,135600000.0,0.001099
1161,Springsteen : Deliver me from nowhere,76000,66000000.0,0.001152
108,Toutes pour une,12000,9900000.0,0.001212


In [820]:
#Quels films sont restés le plus longtemps en top 1 du box office ?

BoxOfficeRang = BoxOffice[BoxOffice['Rang']=='1'].copy()
BoxOfficeRang['Rang'] = BoxOfficeRang['Rang'].astype(int)

result = BoxOfficeRang.groupby('Titre')['Rang'].agg('sum').sort_values(ascending = False)
result[result == result.max()]

Titre
God save the Tuche      4
Minecraft, le film      4
Mufasa : Le roi lion    4
Les bad guys 2          4
Name: Rang, dtype: int64